1. aggregate the data in output 
2. combine with input data 
3. repeat for multiple cars 

4. run regression for feature selection (cut down on features)
5. do some feature engineering (create some interesting interactions?)
6. create regression model using stepwise AIC? 

In [1]:
import numpy as np
import pandas as pd 

In [33]:
## X = data.loc[:, data.columns != 'acsys.CS.LastLap']
## y = data.loc[:, data.columns == 'acsys.CS.LastLap']

In [73]:
# how to select rows from dataframe (multilist)
## t = df.loc[:, df.columns == ('acsys.CS.SpeedKMH', 'mean')]
# we can drop lastlap max and lastlap min because we only need lastlap mean (since all 3 are the same )

### Data Pre-processing (base case: 1 car)


In [6]:
data2_00_output = pd.read_csv('~/Desktop/Redback/data2/data2_00/output.csv')
data2_00_input = pd.read_csv('~/Desktop/Redback/data2/data2_00/input_params.csv')

In [86]:
# We remove some covariates that are described as either 'ignore' or 'unsure' in the data dictionary provided. Also removed columns that were constant values. 
remove_cols = ['acsys.CS.ERSMaxJ', 'acsys.CS.ERSDelivery', 'acsys.CS.ERSHeatCharging', 'acsys.CS.ERSRecovery', # ignore
               'NdSlip_FL', 'NdSlip_FR', 'NdSlip_RL', 'NdSlip_RR', 'TyreSlip_FL', 'TyreSlip_FR', 'TyreSlip_RL', 'TyreSlip_RR', 'DY_FL', 'DY_FR', 'DY_RL', 'DY_RR', # unsure 
               'TyreDirtyLevel_RR', 'TyreDirtyLevel_RL', 'TyreDirtyLevel_FR', 'TyreDirtyLevel_FL', # constant 
               'acsys.CS.DrsEnabled', 'acsys.CS.PerformanceMeter', 'acsys.CS.LapInvalidated', # zeroes
               ] 

data = data2_00_output.drop(remove_cols, axis=1)

# aggregating the output data
df = data.groupby('acsys.CS.LapCount').agg(['mean', 'max', 'min'])


# reshaping input data
data2_00_input.set_index('parameter', inplace = True)
data2_00_input = data2_00_input.transpose()

# replicating rows of input to match number of rows of output 
data2_00_input = data2_00_input.append([data2_00_input]*14,ignore_index=True)


# appending replicated input data to output data
df = df.join(data2_00_input)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


In [87]:
df

,"(acsys.CS.SpeedKMH, mean)","(acsys.CS.SpeedKMH, max)","(acsys.CS.SpeedKMH, min)","(acsys.CS.Gas, mean)","(acsys.CS.Gas, max)","(acsys.CS.Gas, min)","(acsys.CS.Brake, mean)","(acsys.CS.Brake, max)","(acsys.CS.Brake, min)","(acsys.CS.Clutch, mean)",...,suspensions_REAR_STATIC_CAMBER,suspensions_REAR_SPRING_RATE,suspensions_REAR_BUMP_STOP_RATE,suspensions_REAR_PACKER_RANGE,suspensions_REAR_DAMP_BUMP,suspensions_REAR_DAMP_FAST_BUMP,suspensions_REAR_DAMP_REBOUND,suspensions_REAR_DAMP_FAST_REBOUND,tyres_FRONT_PRESSURE_STATIC,tyres_REAR_PRESSURE_STATIC
acsys.CS.LapCount,,,,,,,,,,,,,,,,,,,,,
0,46.824779,84.609711,0.000000,0.719945,1.0,0.000000e+00,0.147455,0.948782,0.000000e+00,0.842030,...,-1.7086,56670.77262,104990.04453,0.10876,1198.9936,503.80323,2101.67425,1153.26332,15.86391,11.37812
1,61.355012,90.476089,39.721592,0.808638,1.0,5.777541e-04,0.042622,0.952429,1.612895e-42,1.000000,...,-1.7086,56670.77262,104990.04453,0.10876,1198.9936,503.80323,2101.67425,1153.26332,15.86391,11.37812
2,42.868842,90.309235,0.002000,0.618188,1.0,0.000000e+00,0.248069,0.966978,1.401298e-45,0.742181,...,-1.7086,56670.77262,104990.04453,0.10876,1198.9936,503.80323,2101.67425,1153.26332,15.86391,11.37812
3,61.096449,90.427589,39.696712,0.811576,1.0,4.314544e-04,0.044044,0.952889,1.904365e-42,1.000000,...,-1.7086,56670.77262,104990.04453,0.10876,1198.9936,503.80323,2101.67425,1153.26332,15.86391,11.37812
4,38.678431,90.291794,0.001395,0.559194,1.0,1.401298e-45,0.306653,0.964213,1.401298e-45,0.669390,...,-1.7086,56670.77262,104990.04453,0.10876,1198.9936,503.80323,2101.67425,1153.26332,15.86391,11.37812
5,60.759023,90.181877,39.782829,0.808622,1.0,8.042427e-05,0.045326,0.945166,2.684888e-42,1.000000,...,-1.7086,56670.77262,104990.04453,0.10876,1198.9936,503.80323,2101.67425,1153.26332,15.86391,11.37812
6,60.949152,89.712265,39.822598,0.809457,1.0,4.134228e-04,0.044219,0.937211,5.351559e-42,1.000000,...,-1.7086,56670.77262,104990.04453,0.10876,1198.9936,503.80323,2101.67425,1153.26332,15.86391,11.37812
7,37.846751,89.822090,0.002026,0.545314,1.0,1.401298e-45,0.316816,0.985332,1.401298e-45,0.652606,...,-1.7086,56670.77262,104990.04453,0.10876,1198.9936,503.80323,2101.67425,1153.26332,15.86391,11.37812
8,60.875029,90.173370,39.801430,0.817968,1.0,7.408310e-05,0.046365,0.951996,4.423899e-42,1.000000,...,-1.7086,56670.77262,104990.04453,0.10876,1198.9936,503.80323,2101.67425,1153.26332,15.86391,11.37812


### Data Pre-processing (multi case: 2 cars)


In [88]:
data2_01_output = pd.read_csv('~/Desktop/Redback/data2/data2_01/output.csv')
data2_01_input = pd.read_csv('~/Desktop/Redback/data2/data2_01/input_params.csv')

In [80]:
# We remove some covariates that are described as either 'ignore' or 'unsure' in the data dictionary provided. Also removed columns that were constant values. 
remove_cols = ['acsys.CS.ERSMaxJ', 'acsys.CS.ERSDelivery', 'acsys.CS.ERSHeatCharging', 'acsys.CS.ERSRecovery', # ignore
               'NdSlip_FL', 'NdSlip_FR', 'NdSlip_RL', 'NdSlip_RR', 'TyreSlip_FL', 'TyreSlip_FR', 'TyreSlip_RL', 'TyreSlip_RR', 'DY_FL', 'DY_FR', 'DY_RL', 'DY_RR', # unsure 
               'TyreDirtyLevel_RR', 'TyreDirtyLevel_RL', 'TyreDirtyLevel_FR', 'TyreDirtyLevel_FL', # constant 
               'acsys.CS.DrsEnabled', 'acsys.CS.PerformanceMeter', 'acsys.CS.LapInvalidated', # zeroes
               ] 

data = data2_01_output.drop(remove_cols, axis=1)

# aggregating the output data
df2 = data.groupby('acsys.CS.LapCount').agg(['mean', 'max', 'min'])


# reshaping input data
data2_01_input.set_index('parameter', inplace = True)
data2_01_input = data2_01_input.transpose()

# replicating rows of input to match number of rows of output 
data2_01_input = data2_01_input.append([data2_01_input]*17,ignore_index=True)

# appending replicated input data to output data
df2 = df2.join(data2_01_input)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


In [81]:
df2

,"(acsys.CS.SpeedKMH, mean)","(acsys.CS.SpeedKMH, max)","(acsys.CS.SpeedKMH, min)","(acsys.CS.Gas, mean)","(acsys.CS.Gas, max)","(acsys.CS.Gas, min)","(acsys.CS.Brake, mean)","(acsys.CS.Brake, max)","(acsys.CS.Brake, min)","(acsys.CS.Clutch, mean)",...,suspensions_REAR_STATIC_CAMBER,suspensions_REAR_SPRING_RATE,suspensions_REAR_BUMP_STOP_RATE,suspensions_REAR_PACKER_RANGE,suspensions_REAR_DAMP_BUMP,suspensions_REAR_DAMP_FAST_BUMP,suspensions_REAR_DAMP_REBOUND,suspensions_REAR_DAMP_FAST_REBOUND,tyres_FRONT_PRESSURE_STATIC,tyres_REAR_PRESSURE_STATIC
acsys.CS.LapCount,,,,,,,,,,,,,,,,,,,,,
0,28.239142,84.235390,0.000000,0.430847,1.0,0.000000e+00,0.429188,0.992120,0.000000e+00,0.508944,...,-0.62184,39718.33932,85258.61539,0.07749,3047.50851,3468.7606,1714.51158,2691.05523,18.67525,12.54306
1,59.538471,89.031715,38.475315,0.793238,1.0,1.334565e-05,0.057318,0.990438,0.000000e+00,1.000000,...,-0.62184,39718.33932,85258.61539,0.07749,3047.50851,3468.7606,1714.51158,2691.05523,18.67525,12.54306
2,59.548165,88.980560,37.982170,0.794097,1.0,1.351976e-05,0.056882,0.992523,0.000000e+00,1.000000,...,-0.62184,39718.33932,85258.61539,0.07749,3047.50851,3468.7606,1714.51158,2691.05523,18.67525,12.54306
3,59.490083,89.022820,38.241676,0.791793,1.0,3.392868e-06,0.058268,0.989590,3.166935e-42,1.000000,...,-0.62184,39718.33932,85258.61539,0.07749,3047.50851,3468.7606,1714.51158,2691.05523,18.67525,12.54306
4,59.518360,89.065117,38.166519,0.792038,1.0,1.173121e-05,0.056770,0.987880,4.402880e-42,1.000000,...,-0.62184,39718.33932,85258.61539,0.07749,3047.50851,3468.7606,1714.51158,2691.05523,18.67525,12.54306
5,59.487243,89.038841,38.326679,0.794688,1.0,1.220172e-05,0.058365,0.992036,3.520062e-42,1.000000,...,-0.62184,39718.33932,85258.61539,0.07749,3047.50851,3468.7606,1714.51158,2691.05523,18.67525,12.54306
6,37.402700,89.097725,0.000869,0.542331,1.0,1.401298e-45,0.320379,0.990787,2.802597e-45,0.663096,...,-0.62184,39718.33932,85258.61539,0.07749,3047.50851,3468.7606,1714.51158,2691.05523,18.67525,12.54306
7,59.266242,89.033142,37.428230,0.790198,1.0,1.128144e-05,0.056976,0.991703,1.357858e-42,1.000000,...,-0.62184,39718.33932,85258.61539,0.07749,3047.50851,3468.7606,1714.51158,2691.05523,18.67525,12.54306
8,59.304266,88.989304,38.367123,0.789794,1.0,8.489615e-06,0.058179,0.992464,3.262223e-42,1.000000,...,-0.62184,39718.33932,85258.61539,0.07749,3047.50851,3468.7606,1714.51158,2691.05523,18.67525,12.54306


In [117]:
df = df.append(df2, ignore_index=True)


In [118]:
df

,"(acsys.CS.SpeedKMH, mean)","(acsys.CS.SpeedKMH, max)","(acsys.CS.SpeedKMH, min)","(acsys.CS.Gas, mean)","(acsys.CS.Gas, max)","(acsys.CS.Gas, min)","(acsys.CS.Brake, mean)","(acsys.CS.Brake, max)","(acsys.CS.Brake, min)","(acsys.CS.Clutch, mean)",...,suspensions_REAR_STATIC_CAMBER,suspensions_REAR_SPRING_RATE,suspensions_REAR_BUMP_STOP_RATE,suspensions_REAR_PACKER_RANGE,suspensions_REAR_DAMP_BUMP,suspensions_REAR_DAMP_FAST_BUMP,suspensions_REAR_DAMP_REBOUND,suspensions_REAR_DAMP_FAST_REBOUND,tyres_FRONT_PRESSURE_STATIC,tyres_REAR_PRESSURE_STATIC
0,46.824779,84.609711,0.000000,0.719945,1.0,0.000000e+00,0.147455,0.948782,0.000000e+00,0.842030,...,-1.70860,56670.77262,104990.04453,0.10876,1198.99360,503.80323,2101.67425,1153.26332,15.86391,11.37812
1,61.355012,90.476089,39.721592,0.808638,1.0,5.777541e-04,0.042622,0.952429,1.612895e-42,1.000000,...,-1.70860,56670.77262,104990.04453,0.10876,1198.99360,503.80323,2101.67425,1153.26332,15.86391,11.37812
2,42.868842,90.309235,0.002000,0.618188,1.0,0.000000e+00,0.248069,0.966978,1.401298e-45,0.742181,...,-1.70860,56670.77262,104990.04453,0.10876,1198.99360,503.80323,2101.67425,1153.26332,15.86391,11.37812
3,61.096449,90.427589,39.696712,0.811576,1.0,4.314544e-04,0.044044,0.952889,1.904365e-42,1.000000,...,-1.70860,56670.77262,104990.04453,0.10876,1198.99360,503.80323,2101.67425,1153.26332,15.86391,11.37812
4,38.678431,90.291794,0.001395,0.559194,1.0,1.401298e-45,0.306653,0.964213,1.401298e-45,0.669390,...,-1.70860,56670.77262,104990.04453,0.10876,1198.99360,503.80323,2101.67425,1153.26332,15.86391,11.37812
5,60.759023,90.181877,39.782829,0.808622,1.0,8.042427e-05,0.045326,0.945166,2.684888e-42,1.000000,...,-1.70860,56670.77262,104990.04453,0.10876,1198.99360,503.80323,2101.67425,1153.26332,15.86391,11.37812
6,60.949152,89.712265,39.822598,0.809457,1.0,4.134228e-04,0.044219,0.937211,5.351559e-42,1.000000,...,-1.70860,56670.77262,104990.04453,0.10876,1198.99360,503.80323,2101.67425,1153.26332,15.86391,11.37812
7,37.846751,89.822090,0.002026,0.545314,1.0,1.401298e-45,0.316816,0.985332,1.401298e-45,0.652606,...,-1.70860,56670.77262,104990.04453,0.10876,1198.99360,503.80323,2101.67425,1153.26332,15.86391,11.37812
8,60.875029,90.173370,39.801430,0.817968,1.0,7.408310e-05,0.046365,0.951996,4.423899e-42,1.000000,...,-1.70860,56670.77262,104990.04453,0.10876,1198.99360,503.80323,2101.67425,1153.26332,15.86391,11.37812
9,61.194076,90.272179,39.671791,0.809430,1.0,9.316421e-05,0.045404,0.955128,2.904892e-42,1.000000,...,-1.70860,56670.77262,104990.04453,0.10876,1198.99360,503.80323,2101.67425,1153.26332,15.86391,11.37812


### Data Pre-processing (final case: multiple cars [2.7z])


In [3]:
# retrieve a list of all the data files from data2_00 to data2_50
data_fold_list = []

i = 0
start = 'data2_'
while i <= 50:
    data_fold = start

    if i <= 9: #00, 01, 02, ..., 09
        data_fold = data_fold + '0' + str(i)
        data_fold_list.append(data_fold)

    else: #10, 11, 12, ..., 50 
        data_fold = data_fold + str(i)
        data_fold_list.append(data_fold)
    
    i = i + 1

print(data_fold_list) # list of all data folders

['data2_00', 'data2_01', 'data2_02', 'data2_03', 'data2_04', 'data2_05', 'data2_06', 'data2_07', 'data2_08', 'data2_09', 'data2_10', 'data2_11', 'data2_12', 'data2_13', 'data2_14', 'data2_15', 'data2_16', 'data2_17', 'data2_18', 'data2_19', 'data2_20', 'data2_21', 'data2_22', 'data2_23', 'data2_24', 'data2_25', 'data2_26', 'data2_27', 'data2_28', 'data2_29', 'data2_30', 'data2_31', 'data2_32', 'data2_33', 'data2_34', 'data2_35', 'data2_36', 'data2_37', 'data2_38', 'data2_39', 'data2_40', 'data2_41', 'data2_42', 'data2_43', 'data2_44', 'data2_45', 'data2_46', 'data2_47', 'data2_48', 'data2_49', 'data2_50']


In [4]:
# mass importing of data files (output and input) for each data folder 
data2_output = [] # data2_00_o, data2_01_o, ... data2_50_o
data2_input = [] # data2_00_i, data2_01_i, ... data2_50_i

i = 0 
front = '~/Desktop/Redback/data2/'
backi = '/input_params.csv'
backo = '/output.csv'
while i <= 50: 
    curro = front
    curri = front

    curro = curro + data_fold_list[i] + backo 
    curri = curri + data_fold_list[i] + backi

    data2_output.append(curro)
    data2_input.append(curri)
    i = i + 1

In [5]:
i = 0
#data_prev = pd.DataFrame
lap_count = []
while i <= 50: 
    data2_out_curr = pd.read_csv(data2_output[i])
    data2_inp_curr = pd.read_csv(data2_input[i])

    if i == 0: # base case 
        # We remove some covariates that are described as either 'ignore' or 'unsure' in the data dictionary provided. Also removed columns that were constant values. 
        remove_cols = ['acsys.CS.ERSMaxJ', 'acsys.CS.ERSDelivery', 'acsys.CS.ERSHeatCharging', 'acsys.CS.ERSRecovery', # ignore
                    'NdSlip_FL', 'NdSlip_FR', 'NdSlip_RL', 'NdSlip_RR', 'TyreSlip_FL', 'TyreSlip_FR', 'TyreSlip_RL', 'TyreSlip_RR', 'DY_FL', 'DY_FR', 'DY_RL', 'DY_RR', # unsure 
                    'TyreDirtyLevel_RR', 'TyreDirtyLevel_RL', 'TyreDirtyLevel_FR', 'TyreDirtyLevel_FL', # constant 
                    'acsys.CS.DrsEnabled', 'acsys.CS.PerformanceMeter', 'acsys.CS.LapInvalidated', # zeroes
                    ] 

        data = data2_out_curr.drop(remove_cols, axis=1)

        # aggregating the output data
        df = data.groupby('acsys.CS.LapCount').agg(['mean', 'max', 'min'])
        lap_count.append(df.shape[0] - 1) 

        # reshaping input data
        data2_inp_curr.set_index('parameter', inplace = True)
        data2_inp_curr = data2_inp_curr.transpose()

        # replicating rows of input to match number of rows of output 
        data2_inp_curr = data2_inp_curr.append([data2_inp_curr]*lap_count[i],ignore_index=True)

        # appending replicated input data to output data
        df = df.join(data2_inp_curr)

    else: # multiple case 
        # We remove some covariates that are described as either 'ignore' or 'unsure' in the data dictionary provided. Also removed columns that were constant values. 
        remove_cols = ['acsys.CS.ERSMaxJ', 'acsys.CS.ERSDelivery', 'acsys.CS.ERSHeatCharging', 'acsys.CS.ERSRecovery', # ignore
                    'NdSlip_FL', 'NdSlip_FR', 'NdSlip_RL', 'NdSlip_RR', 'TyreSlip_FL', 'TyreSlip_FR', 'TyreSlip_RL', 'TyreSlip_RR', 'DY_FL', 'DY_FR', 'DY_RL', 'DY_RR', # unsure 
                    'TyreDirtyLevel_RR', 'TyreDirtyLevel_RL', 'TyreDirtyLevel_FR', 'TyreDirtyLevel_FL', # constant 
                    'acsys.CS.DrsEnabled', 'acsys.CS.PerformanceMeter', 'acsys.CS.LapInvalidated', # zeroes
                    ] 

        data = data2_out_curr.drop(remove_cols, axis=1)

        # aggregating the output data
        df2 = data.groupby('acsys.CS.LapCount').agg(['mean', 'max', 'min'])
        lap_count.append(df2.shape[0] - 1) 

        # reshaping input data
        data2_inp_curr.set_index('parameter', inplace = True)
        data2_inp_curr = data2_inp_curr.transpose()

        # replicating rows of input to match number of rows of output 
        data2_inp_curr = data2_inp_curr.append([data2_inp_curr]*lap_count[i],ignore_index=True)

        # appending replicated input data to output data
        df2 = df2.join(data2_inp_curr)

        df = df.append(df2, ignore_index=True)


    i = i + 1 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


Takes about 57.6 seconds to run the above 

In [6]:
df

,"(acsys.CS.SpeedKMH, mean)","(acsys.CS.SpeedKMH, max)","(acsys.CS.SpeedKMH, min)","(acsys.CS.Gas, mean)","(acsys.CS.Gas, max)","(acsys.CS.Gas, min)","(acsys.CS.Brake, mean)","(acsys.CS.Brake, max)","(acsys.CS.Brake, min)","(acsys.CS.Clutch, mean)",...,suspensions_REAR_STATIC_CAMBER,suspensions_REAR_SPRING_RATE,suspensions_REAR_BUMP_STOP_RATE,suspensions_REAR_PACKER_RANGE,suspensions_REAR_DAMP_BUMP,suspensions_REAR_DAMP_FAST_BUMP,suspensions_REAR_DAMP_REBOUND,suspensions_REAR_DAMP_FAST_REBOUND,tyres_FRONT_PRESSURE_STATIC,tyres_REAR_PRESSURE_STATIC
0,46.824779,84.609711,0.000000,0.719945,1.0,0.000000e+00,0.147455,0.948782,0.000000e+00,0.842030,...,-1.70860,56670.77262,104990.04453,0.10876,1198.99360,503.80323,2101.67425,1153.26332,15.86391,11.37812
1,61.355012,90.476089,39.721592,0.808638,1.0,5.777541e-04,0.042622,0.952429,1.612895e-42,1.000000,...,-1.70860,56670.77262,104990.04453,0.10876,1198.99360,503.80323,2101.67425,1153.26332,15.86391,11.37812
2,42.868842,90.309235,0.002000,0.618188,1.0,0.000000e+00,0.248069,0.966978,1.401298e-45,0.742181,...,-1.70860,56670.77262,104990.04453,0.10876,1198.99360,503.80323,2101.67425,1153.26332,15.86391,11.37812
3,61.096449,90.427589,39.696712,0.811576,1.0,4.314544e-04,0.044044,0.952889,1.904365e-42,1.000000,...,-1.70860,56670.77262,104990.04453,0.10876,1198.99360,503.80323,2101.67425,1153.26332,15.86391,11.37812
4,38.678431,90.291794,0.001395,0.559194,1.0,1.401298e-45,0.306653,0.964213,1.401298e-45,0.669390,...,-1.70860,56670.77262,104990.04453,0.10876,1198.99360,503.80323,2101.67425,1153.26332,15.86391,11.37812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845,55.165388,77.630219,39.373135,0.772242,1.0,1.075203e-04,0.020489,0.926061,1.401298e-45,1.000000,...,-1.61895,55360.22156,70589.12132,0.08101,3402.35898,602.93357,3530.31009,2314.02937,6.31958,17.23205
846,55.756241,77.701698,39.736221,0.780780,1.0,1.116195e-04,0.021695,0.815666,1.401298e-45,1.000000,...,-1.61895,55360.22156,70589.12132,0.08101,3402.35898,602.93357,3530.31009,2314.02937,6.31958,17.23205
847,55.681054,77.626526,39.131283,0.780174,1.0,1.213556e-04,0.022081,0.821020,1.401298e-45,1.000000,...,-1.61895,55360.22156,70589.12132,0.08101,3402.35898,602.93357,3530.31009,2314.02937,6.31958,17.23205
848,43.349198,77.726631,0.000507,0.668329,1.0,1.401298e-45,0.138203,0.850000,1.401298e-45,0.829549,...,-1.61895,55360.22156,70589.12132,0.08101,3402.35898,602.93357,3530.31009,2314.02937,6.31958,17.23205


In [7]:
# exporting the master dataframe that we will be using. This dataframe is data taken from data2.7z of 50 cars 
df.to_csv('master_df2.csv')

### Data Pre-processing (final case: multiple cars [1.7z])


In [8]:
# retrieve a list of all the data files from data1_00 to data1_32
data_fold_list = []

i = 0
start = 'data1_'
while i <= 32:
    data_fold = start

    if i <= 9: #00, 01, 02, ..., 09
        data_fold = data_fold + '0' + str(i)
        data_fold_list.append(data_fold)

    else: #10, 11, 12, ..., 32
        data_fold = data_fold + str(i)
        data_fold_list.append(data_fold)
    
    i = i + 1

print(data_fold_list) # list of all data folders

['data1_00', 'data1_01', 'data1_02', 'data1_03', 'data1_04', 'data1_05', 'data1_06', 'data1_07', 'data1_08', 'data1_09', 'data1_10', 'data1_11', 'data1_12', 'data1_13', 'data1_14', 'data1_15', 'data1_16', 'data1_17', 'data1_18', 'data1_19', 'data1_20', 'data1_21', 'data1_22', 'data1_23', 'data1_24', 'data1_25', 'data1_26', 'data1_27', 'data1_28', 'data1_29', 'data1_30', 'data1_31', 'data1_32']


In [9]:
# mass importing of data files (output and input) for each data folder 
data1_output = [] 
data1_input = []

i = 0 
front = '~/Desktop/Redback/data1/'
backi = '/input_params.csv'
backo = '/output.csv'
while i <= 32: 
    curro = front
    curri = front

    curro = curro + data_fold_list[i] + backo 
    curri = curri + data_fold_list[i] + backi

    data1_output.append(curro)
    data1_input.append(curri)
    i = i + 1

In [10]:
i = 0
lap_count = []
while i <= 32: 
    data1_out_curr = pd.read_csv(data1_output[i])
    data1_inp_curr = pd.read_csv(data1_input[i])

    if i == 0: # base case 
        # We remove some covariates that are described as either 'ignore' or 'unsure' in the data dictionary provided. Also removed columns that were constant values. 
        remove_cols = ['acsys.CS.ERSMaxJ', 'acsys.CS.ERSDelivery', 'acsys.CS.ERSHeatCharging', 'acsys.CS.ERSRecovery', # ignore
                    'NdSlip_FL', 'NdSlip_FR', 'NdSlip_RL', 'NdSlip_RR', 'TyreSlip_FL', 'TyreSlip_FR', 'TyreSlip_RL', 'TyreSlip_RR', 'DY_FL', 'DY_FR', 'DY_RL', 'DY_RR', # unsure 
                    'TyreDirtyLevel_RR', 'TyreDirtyLevel_RL', 'TyreDirtyLevel_FR', 'TyreDirtyLevel_FL', # constant 
                    'acsys.CS.DrsEnabled', 'acsys.CS.PerformanceMeter', 'acsys.CS.LapInvalidated', # zeroes
                    ] 

        data = data1_out_curr.drop(remove_cols, axis=1)

        # aggregating the output data
        df = data.groupby('acsys.CS.LapCount').agg(['mean', 'max', 'min'])
        lap_count.append(df.shape[0] - 1) 

        # reshaping input data
        data1_inp_curr.set_index('parameter', inplace = True)
        data1_inp_curr = data1_inp_curr.transpose()

        # replicating rows of input to match number of rows of output 
        data1_inp_curr = data1_inp_curr.append([data1_inp_curr]*lap_count[i],ignore_index=True)

        # appending replicated input data to output data
        df = df.join(data1_inp_curr)

    else: # multiple case 
        # We remove some covariates that are described as either 'ignore' or 'unsure' in the data dictionary provided. Also removed columns that were constant values. 
        remove_cols = ['acsys.CS.ERSMaxJ', 'acsys.CS.ERSDelivery', 'acsys.CS.ERSHeatCharging', 'acsys.CS.ERSRecovery', # ignore
                    'NdSlip_FL', 'NdSlip_FR', 'NdSlip_RL', 'NdSlip_RR', 'TyreSlip_FL', 'TyreSlip_FR', 'TyreSlip_RL', 'TyreSlip_RR', 'DY_FL', 'DY_FR', 'DY_RL', 'DY_RR', # unsure 
                    'TyreDirtyLevel_RR', 'TyreDirtyLevel_RL', 'TyreDirtyLevel_FR', 'TyreDirtyLevel_FL', # constant 
                    'acsys.CS.DrsEnabled', 'acsys.CS.PerformanceMeter', 'acsys.CS.LapInvalidated', # zeroes
                    ] 

        data = data1_out_curr.drop(remove_cols, axis=1)

        # aggregating the output data
        df2 = data.groupby('acsys.CS.LapCount').agg(['mean', 'max', 'min'])
        lap_count.append(df2.shape[0] - 1) 

        # reshaping input data
        data1_inp_curr.set_index('parameter', inplace = True)
        data1_inp_curr = data1_inp_curr.transpose()

        # replicating rows of input to match number of rows of output 
        data1_inp_curr = data1_inp_curr.append([data1_inp_curr]*lap_count[i],ignore_index=True)

        # appending replicated input data to output data
        df2 = df2.join(data1_inp_curr)

        df = df.append(df2, ignore_index=True)


    i = i + 1 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


In [11]:
df

,"(acsys.CS.SpeedKMH, mean)","(acsys.CS.SpeedKMH, max)","(acsys.CS.SpeedKMH, min)","(acsys.CS.Gas, mean)","(acsys.CS.Gas, max)","(acsys.CS.Gas, min)","(acsys.CS.Brake, mean)","(acsys.CS.Brake, max)","(acsys.CS.Brake, min)","(acsys.CS.Clutch, mean)",...,suspensions_REAR_STATIC_CAMBER,suspensions_REAR_SPRING_RATE,suspensions_REAR_BUMP_STOP_RATE,suspensions_REAR_PACKER_RANGE,suspensions_REAR_DAMP_BUMP,suspensions_REAR_DAMP_FAST_BUMP,suspensions_REAR_DAMP_REBOUND,suspensions_REAR_DAMP_FAST_REBOUND,tyres_FRONT_PRESSURE_STATIC,tyres_REAR_PRESSURE_STATIC
0,19.533266,86.119110,0.000000,0.303412,1.0,0.000000e+00,0.561093,0.955766,0.000000e+00,0.348155,...,-2.35767,56648.86635,124979.65621,0.08939,1161.13758,3342.89845,3747.30276,3478.35963,9.18841,14.31058
1,60.124537,91.812538,40.066288,0.824581,1.0,3.085351e-04,0.052606,0.951633,0.000000e+00,1.000000,...,-2.35767,56648.86635,124979.65621,0.08939,1161.13758,3342.89845,3747.30276,3478.35963,9.18841,14.31058
2,59.912932,91.628632,38.526600,0.816105,1.0,2.976748e-04,0.054815,0.953510,1.066388e-41,1.000000,...,-2.35767,56648.86635,124979.65621,0.08939,1161.13758,3342.89845,3747.30276,3478.35963,9.18841,14.31058
3,35.412222,91.713875,0.002218,0.527852,1.0,1.401298e-45,0.349217,0.975612,1.401298e-45,0.622166,...,-2.35767,56648.86635,124979.65621,0.08939,1161.13758,3342.89845,3747.30276,3478.35963,9.18841,14.31058
4,59.446034,91.466446,38.588680,0.813708,1.0,2.992767e-04,0.055115,0.934650,7.328791e-42,1.000000,...,-2.35767,56648.86635,124979.65621,0.08939,1161.13758,3342.89845,3747.30276,3478.35963,9.18841,14.31058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086,30.049857,86.521111,0.001136,0.464617,1.0,1.401298e-45,0.397413,0.850000,1.401298e-45,0.549677,...,-1.54650,35590.17408,78945.44905,0.06733,2623.27446,3429.20750,1820.40333,1660.40021,14.44796,15.89340
1087,57.931751,86.124931,37.967102,0.817472,1.0,2.272607e-04,0.032326,0.788827,0.000000e+00,1.000000,...,-1.54650,35590.17408,78945.44905,0.06733,2623.27446,3429.20750,1820.40333,1660.40021,14.44796,15.89340
1088,57.560393,86.559700,38.140427,0.812400,1.0,2.821523e-05,0.032975,0.938923,0.000000e+00,1.000000,...,-1.54650,35590.17408,78945.44905,0.06733,2623.27446,3429.20750,1820.40333,1660.40021,14.44796,15.89340
1089,59.002506,86.405434,38.245289,0.792537,1.0,5.037030e-09,0.034070,0.782344,1.401298e-45,1.000000,...,-1.54650,35590.17408,78945.44905,0.06733,2623.27446,3429.20750,1820.40333,1660.40021,14.44796,15.89340


In [12]:
df.to_csv('master_df1.csv')

### Data Pre-processing (The Ultimate Master Data Frame [2.7z + 1.7z])


In [2]:
df2_7 = pd.read_csv('~/Desktop/Redback/master_df2.csv')
df1_7 = pd.read_csv('~/Desktop/Redback/master_df1.csv')

Ultimate_df = df2_7.append(df1_7, ignore_index=True)

remove_cols = ['(\'acsys.CS.LastLap\', \'max\')', '(\'acsys.CS.LastLap\', \'min\')', 
               '(\'acsys.CS.BestLap\', \'max\')', '(\'acsys.CS.BestLap\', \'min\')', 
               '(\'acsys.CS.RaceFinished\', \'max\')', '(\'acsys.CS.RaceFinished\', \'min\')', '(\'acsys.CS.RaceFinished\', \'mean\')', 
               '(\'acsys.CS.Caster\', \'max\')', '(\'acsys.CS.Caster\', \'min\')', '(\'acsys.CS.Caster\', \'mean\')',

               'Unnamed: 0'] 
Ultimate_df = Ultimate_df.drop(remove_cols, axis=1)

Ultimate_df.rename(columns={'(\'acsys.CS.LapTime\', \'mean\')': 'LapTime_mean', 
                            '(\'acsys.CS.LapTime\', \'max\')': 'LapTime_max', 
                            '(\'acsys.CS.LapTime\', \'min\')': 'LapTime_min', 
                            '(\'acsys.CS.SpeedKMH\', \'mean\')': 'SpeedKMH_mean',
                            '(\'acsys.CS.SpeedKMH\', \'max\')': 'SpeedKMH_max',
                            '(\'acsys.CS.SpeedKMH\', \'min\')': 'SpeedKMH_min',
                            '(\'acsys.CS.Gas\', \'mean\')': 'Gas_mean',
                            '(\'acsys.CS.Gas\', \'max\')': 'Gas_max',
                            '(\'acsys.CS.Gas\', \'min\')': 'Gas_min',
                            '(\'acsys.CS.Brake\', \'mean\')': 'Brake_mean',
                            '(\'acsys.CS.Brake\', \'max\')': 'Brake_max',
                            '(\'acsys.CS.Brake\', \'min\')': 'Brake_min',
                            '(\'acsys.CS.Clutch\', \'mean\')': 'Clutch_mean',
                            '(\'acsys.CS.Clutch\', \'max\')': 'Clutch_max',
                            '(\'acsys.CS.Clutch\', \'min\')': 'Clutch_min',
                            '(\'acsys.CS.Gear\', \'mean\')': 'Gear_mean',
                            '(\'acsys.CS.Gear\', \'max\')': 'Gear_max',
                            '(\'acsys.CS.Gear\', \'min\')': 'Gear_min',
                            '(\'acsys.CS.BestLap\', \'mean\')': 'Gear_mean',
                            '(\'acsys.CS.CGHeight\', \'mean\')': 'CGHeight_mean',
                            '(\'acsys.CS.CGHeight\', \'max\')': 'CGHeight_max',
                            '(\'acsys.CS.CGHeight\', \'min\')': 'CGHeight_min',
                            '(\'acsys.CS.DriveTrainSpeed\', \'mean\')': 'DriveTrainSpeed_mean',
                            '(\'acsys.CS.DriveTrainSpeed\', \'max\')': 'DriveTrainSpeed_max',
                            '(\'acsys.CS.DriveTrainSpeed\', \'min\')': 'DriveTrainSpeed_min',
                            '(\'acsys.CS.RPM\', \'mean\')': 'RPM_mean',
                            '(\'acsys.CS.RPM\', \'max\')': 'RPM_max',
                            '(\'acsys.CS.RPM\', \'min\')': 'RPM_min',
                            }, inplace=True)


# removing pit laps (initially it was > 75000)
Ultimate_df = Ultimate_df.drop(Ultimate_df[Ultimate_df.LapTime_max > 60000].index)

Ultimate_df.to_csv('Ultimate_df.csv')

In [3]:
Ultimate_df

,SpeedKMH_mean,SpeedKMH_max,SpeedKMH_min,Gas_mean,Gas_max,Gas_min,Brake_mean,Brake_max,Brake_min,Clutch_mean,...,suspensions_REAR_STATIC_CAMBER,suspensions_REAR_SPRING_RATE,suspensions_REAR_BUMP_STOP_RATE,suspensions_REAR_PACKER_RANGE,suspensions_REAR_DAMP_BUMP,suspensions_REAR_DAMP_FAST_BUMP,suspensions_REAR_DAMP_REBOUND,suspensions_REAR_DAMP_FAST_REBOUND,tyres_FRONT_PRESSURE_STATIC,tyres_REAR_PRESSURE_STATIC
0,46.824779,84.609711,0.000000,0.719945,1.0,0.000000e+00,0.147455,0.948782,0.000000e+00,0.84203,...,-1.7086,56670.77262,104990.04453,0.10876,1198.99360,503.80323,2101.67425,1153.26332,15.86391,11.37812
1,61.355012,90.476089,39.721592,0.808638,1.0,5.777541e-04,0.042622,0.952429,1.612895e-42,1.00000,...,-1.7086,56670.77262,104990.04453,0.10876,1198.99360,503.80323,2101.67425,1153.26332,15.86391,11.37812
3,61.096449,90.427589,39.696712,0.811576,1.0,4.314544e-04,0.044044,0.952889,1.904365e-42,1.00000,...,-1.7086,56670.77262,104990.04453,0.10876,1198.99360,503.80323,2101.67425,1153.26332,15.86391,11.37812
5,60.759023,90.181877,39.782829,0.808622,1.0,8.042427e-05,0.045326,0.945166,2.684888e-42,1.00000,...,-1.7086,56670.77262,104990.04453,0.10876,1198.99360,503.80323,2101.67425,1153.26332,15.86391,11.37812
6,60.949152,89.712265,39.822598,0.809457,1.0,4.134228e-04,0.044219,0.937211,5.351559e-42,1.00000,...,-1.7086,56670.77262,104990.04453,0.10876,1198.99360,503.80323,2101.67425,1153.26332,15.86391,11.37812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1935,58.521987,86.206688,38.243599,0.823782,1.0,2.921104e-04,0.033674,0.801815,1.401298e-45,1.00000,...,-1.5465,35590.17408,78945.44905,0.06733,2623.27446,3429.20750,1820.40333,1660.40021,14.44796,15.89340
1937,57.931751,86.124931,37.967102,0.817472,1.0,2.272607e-04,0.032326,0.788827,0.000000e+00,1.00000,...,-1.5465,35590.17408,78945.44905,0.06733,2623.27446,3429.20750,1820.40333,1660.40021,14.44796,15.89340
1938,57.560393,86.559700,38.140427,0.812400,1.0,2.821523e-05,0.032975,0.938923,0.000000e+00,1.00000,...,-1.5465,35590.17408,78945.44905,0.06733,2623.27446,3429.20750,1820.40333,1660.40021,14.44796,15.89340
1939,59.002506,86.405434,38.245289,0.792537,1.0,5.037030e-09,0.034070,0.782344,1.401298e-45,1.00000,...,-1.5465,35590.17408,78945.44905,0.06733,2623.27446,3429.20750,1820.40333,1660.40021,14.44796,15.89340
